[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/georgianpartners/Multimodal-Toolkit/blob/master/notebooks/text_w_tabular_classification.ipynb)

# Training a BertWithTabular Model for Clothing Review Recommendation Prediction

This guide closely follows the [example](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/trainer/01_text_classification.ipynb#scrollTo=bwl3I_VGAZXb) from HuggingFace for text classification on the GLUE dataset.

Install `multimodal-transformers` and `datasets` (for getting a dataset).

In [1]:
!pip install multimodal-transformers
!pip install datasets
!pip install tensorboard

## All other imports are here:

In [1]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoConfig, Trainer, EvalPrediction, set_seed
from transformers.training_args import TrainingArguments

import sys

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular
from multimodal_transformers.multimodal_arguments import (
    ModelArguments,
    MultimodalDataTrainingArguments,
)

logging.basicConfig(level=logging.INFO)
os.environ["COMET_MODE"] = "DISABLED"

/Users/akashsaravanan/opt/anaconda3/envs/multimodal_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Important Note:** If you run this notebook on Google Colab and face any issue with numpy, please restart the kernel once and try again. Refer to this [issue](https://github.com/georgian-io/Multimodal-Toolkit/issues/71) for more information.

## Dataset

Our dataset is the [Womens Clothing E-Commerce Reviews](https://huggingface.co/datasets/Censius-AI/ECommerce-Women-Clothing-Reviews) dataset. It contains reviews written by customers about clothing items as well as whether they recommend the data or not. We download the dataset here.

In [2]:
from datasets import load_dataset

dataset = load_dataset("Censius-AI/ECommerce-Women-Clothing-Reviews")

#### Let us take a look at what the dataset looks like

In [3]:
data_df = dataset['train'].to_pandas()
data_df.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,None,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,None,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


We see that the data contains both text in the `Review Text` and `Title` column as well as tabular features in the `Division Name`, `Department Name`, and `Class Name` columns. 

In [4]:
data_df.describe(include=object)

,Title,Review Text,Division Name,Department Name,Class Name
count,19676,22641,23472,23472,23472
unique,13993,22634,3,6,20
top,Love it!,Perfect fit and i've gotten so many compliment...,General,Tops,Dresses
freq,136,3,13850,10468,6319


In [5]:
data_df.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


We split our data into 8:1:1 training splits. We also save our splits to `train.csv`, `val.csv`, and `test.csv` as this is the format our dataloader requires.

In [6]:
train_df, val_df, test_df = np.split(data_df.sample(frac=1), [int(.8*len(data_df)), int(.9 * len(data_df))])
print('Num examples train-val-test')
print(len(train_df), len(val_df), len(test_df))
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

/Users/akashsaravanan/opt/anaconda3/envs/multimodal_test/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Num examples train-val-test
18788 2349 2349


## We then our Experiment Parameters

For model we can specify any supported HuggingFace model classes (see README for more details) as well as any AutoModel that are from the supported model classes. For the data specifications, we need to specify a dictionary that specifies which columns are the `text` columns, `numerical feature` columns, `categorical feature` column, and the `label` column. If we are doing classification, we can also specify what each of the labels means in the label column through the `label list`. We can also specifiy these columns using a path to a json file with the argument `column_info_path` to `MultimodalDataTrainingArguments`.

In [7]:
text_cols = ["Title", "Review Text"]
cat_cols = ["Clothing ID", "Division Name", "Department Name", "Class Name"]
numerical_cols = ["Rating", "Age", "Positive Feedback Count"]

column_info_dict = {
    "text_cols": text_cols,
    "num_cols": numerical_cols,
    "cat_cols": cat_cols,
    "label_col": "Recommended IND",
    "label_list": ["Not Recommended", "Recommended"],
}


model_args = ModelArguments(model_name_or_path="bert-base-uncased")

data_args = MultimodalDataTrainingArguments(
    data_path=".",
    combine_feat_method="individual_mlps_on_cat_and_numerical_feats_then_concat",
    column_info=column_info_dict,
    task="classification",
    categorical_encode_type="binary",
    categorical_handle_na=True,
    categorical_na_value="Unknown",
    ohe_handle_unknown="error",
    numerical_transformer_method="yeo_johnson",
    numerical_handle_na=True,
    numerical_how_handle_na="median",
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=1,
    logging_steps=25,
    eval_steps=250,
)

set_seed(training_args.seed)

## Now we can load our model and data. 
### We first instantiate our HuggingFace tokenizer
This is needed to prepare our custom torch dataset. See `torch_dataset.py` for details.

In [8]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  bert-base-uncased


### Load dataset csvs to torch datasets
The function `load_data_from_folder` expects a path to a folder that contains `train.csv`, `test.csv`, and/or `val.csv` containing the respective split datasets. 

In [9]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info["text_cols"],
    tokenizer,
    label_col=data_args.column_info["label_col"],
    label_list=data_args.column_info["label_list"],
    categorical_cols=data_args.column_info["cat_cols"],
    numerical_cols=data_args.column_info["num_cols"],
    sep_text_token_str=tokenizer.sep_token,
)

INFO:multimodal_transformers.data.load_data:Text columns: ['Title', 'Review Text']
INFO:multimodal_transformers.data.load_data:Raw text example: Such a cute top! can be dressed up or down. i wore it on sting break and had many compliments.
INFO:multimodal_transformers.data.load_data:Text columns: ['Title', 'Review Text']
INFO:multimodal_transformers.data.load_data:Raw text example: Great dress for spring [SEP] Great dress that can be dressed up for work day or even dressed down for a nice weekend day.
i have it in black but thinking of buying in the yellow/orange color too.
INFO:multimodal_transformers.data.load_data:Text columns: ['Title', 'Review Text']
INFO:multimodal_transformers.data.load_data:Raw text example: Loved this top from the moment i saw it. i was bummed that this top wasn't available in petite sizes, but since it was my birthday i thought i would treat myself. i purchased the xs. after receiving it and trying it on, it was big under the arms and waist area, i kind of ex

In [10]:
num_labels = len(np.unique(train_dataset.labels))
num_labels

2

In [11]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
tabular_config = TabularConfig(
    num_labels=num_labels,
    cat_feat_dim=train_dataset.cat_feats.shape[1],
    numerical_feat_dim=train_dataset.numerical_feats.shape[1],
    **vars(data_args)
)
config.tabular_config = tabular_config

In [12]:
model = AutoModelWithTabular.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    config=config,
    cache_dir=model_args.cache_dir,
)

Some weights of BertWithTabular were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'tabular_classifier.bias', 'tabular_classifier.weight', 'tabular_combiner.cat_mlp.layers.0.bias', 'tabular_combiner.cat_mlp.layers.0.weight', 'tabular_combiner.num_mlp.bn.0.bias', 'tabular_combiner.num_mlp.bn.0.num_batches_tracked', 'tabular_combiner.num_mlp.bn.0.running_mean', 'tabular_combiner.num_mlp.bn.0.running_var', 'tabular_combiner.num_mlp.bn.0.weight', 'tabular_combiner.num_mlp.layers.0.bias', 'tabular_combiner.num_mlp.layers.0.weight', 'tabular_combiner.num_mlp.layers.1.bias', 'tabular_combiner.num_mlp.layers.1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### We need to define a task-specific way of computing relevant metrics:

In [13]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)


def calc_classification_metrics(p: EvalPrediction):
    predictions = p.predictions[0]
    pred_labels = np.argmax(predictions, axis=1)
    pred_scores = softmax(predictions, axis=1)[:, 1]
    labels = p.label_ids
    if len(np.unique(labels)) == 2:  # binary classification
        roc_auc_pred_score = roc_auc_score(labels, pred_scores)
        precisions, recalls, thresholds = precision_recall_curve(labels, pred_scores)
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        fscore[np.isnan(fscore)] = 0
        ix = np.argmax(fscore)
        threshold = thresholds[ix].item()
        pr_auc = auc(recalls, precisions)
        tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
        result = {
            "roc_auc": roc_auc_pred_score,
            "threshold": threshold,
            "pr_auc": pr_auc,
            "recall": recalls[ix].item(),
            "precision": precisions[ix].item(),
            "f1": fscore[ix].item(),
            "tn": tn.item(),
            "fp": fp.item(),
            "fn": fn.item(),
            "tp": tp.item(),
        }
    else:
        acc = (pred_labels == labels).mean()
        f1 = f1_score(y_true=labels, y_pred=pred_labels)
        result = {
            "acc": acc,
            "f1": f1,
            "acc_and_f1": (acc + f1) / 2,
            "mcc": matthews_corrcoef(labels, pred_labels),
        }

    return result

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)

## Launching the training is as simple is doing trainer.train() 🤗

Note: We set `training_args.max_steps` to 10 just for the sake of a faster demo. This isn't included in this starter code to prevent folks from getting confused.

In [15]:
%%time
trainer.train()

100%|██████████| 2/2 [00:30<00:00, 15.38s/it]

{'train_runtime': 30.7548, 'train_samples_per_second': 1.626, 'train_steps_per_second': 0.065, 'train_loss': 0.6142770051956177, 'epoch': 1.0}
CPU times: user 1min 26s, sys: 13.7 s, total: 1min 40s
Wall time: 31 s


TrainOutput(global_step=2, training_loss=0.6142770051956177, metrics={'train_runtime': 30.7548, 'train_samples_per_second': 1.626, 'train_steps_per_second': 0.065, 'total_flos': 3655475943300.0, 'train_loss': 0.6142770051956177, 'epoch': 1.0})

### Evaluating on the validation data

In [16]:
%%time
trainer.evaluate(eval_dataset=torch.utils.data.random_split(val_dataset, [0.01, 0.99])[0])

  0%|          | 0/1 [00:00<?, ?it/s]/Users/akashsaravanan/opt/anaconda3/envs/multimodal_test/lib/python3.10/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 42.27it/s]

CPU times: user 945 ms, sys: 48.6 ms, total: 993 ms
Wall time: 462 ms


{'eval_loss': 0.39344149827957153,
 'eval_acc': 1.0,
 'eval_f1': 1.0,
 'eval_acc_and_f1': 1.0,
 'eval_mcc': 0.0,
 'eval_runtime': 0.4567,
 'eval_samples_per_second': 2.19,
 'eval_steps_per_second': 2.19,
 'epoch': 1.0}

# Inference

Below is an example of running inference after training the model. We use a test batch of size 32. If you want to test a single example, ensure you still pass it as a batch to the model.

In [17]:
test_batch = test_dataset[:32]
test_batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'cat_feats', 'numerical_feats'])

In [18]:
model.eval()
with torch.no_grad():
    _, logits, classifier_outputs = model(
        test_batch["input_ids"],
        attention_mask=test_batch["attention_mask"],
        token_type_ids=test_batch["token_type_ids"],
        cat_feats=test_batch["cat_feats"],
        numerical_feats=test_batch["numerical_feats"],
    )

In [19]:
acc = torch.sum(logits.argmax(axis=1) == test_batch["labels"]) / logits.shape[0]
print(f"Accuracy: {acc}")

Accuracy: 0.78125
